In [99]:
from collections import defaultdict, Counter
from tqdm import tqdm
import csv
import os
import pandas as pd

In [29]:
ROOT = '/mnt/data0/lucy/manosphere/'
ANN_FILE = ROOT + 'data/ann_sig_entities.csv'
COREF_LOGS = '/mnt/data0/dtadimeti/manosphere/logs/'
COREF_REDDIT = COREF_LOGS + 'coref_reddit/'
COREF_FORUMS = COREF_LOGS + 'coref_forums/'
COREF_CONTROL = COREF_LOGS + 'coref_control/'
COREF_RESULTS = ROOT + 'logs/coref_results/'

In [26]:
# Get subreddit categories 
categories = defaultdict(str)
categories_rev = defaultdict(list)
with open(SUB_META, 'r') as infile: 
    reader = csv.DictReader(infile)
    for row in reader: 
        name = row['Subreddit'].strip().lower()
        if name.startswith('/r/'): name = name[3:]
        if name.startswith('r/'): name = name[2:]
        if name.endswith('/'): name = name[:-1]
        categories[name] = row['Category after majority agreement']
        categories_rev[row['Category after majority agreement']].append(name)

In [101]:
df = pd.read_csv(COREF_RESULTS + 'coref_reddit_df.csv')
df.community.unique()

array(['PUA', 'MRA', 'Incels', 'TRP', 'MGTOW', 'FDS', 'Femcels'],
      dtype=object)

Additional errors.

In [97]:
df[df['community'] == '']

,month,community,word,fem,masc,neut,it,you


In [102]:
print(sum(df['fem'].to_list()))
print(sum(df['masc'].to_list()))

5145
8477


In [105]:
1 - (sum(df['fem'].to_list()) / (sum(df['fem'].to_list()) + sum(df['masc'].to_list())))

0.6223021582733813

### Popular fem words in reddit

In [80]:
def show_top_fem(cat): 
    cat_df = df[df.category == cat]
    cat_totals = cat_df.groupby('word').sum()
    cat_totals['total'] = cat_totals['fem'] + cat_totals['masc'] 
    # filter to only those that appear at least 10 times as she or he
    cat_totals = cat_totals[cat_totals['total'] > 10] 
    cat_totals['fem_frac'] = cat_totals['fem'] / (cat_totals['fem'] + cat_totals['masc'])
    cat_fem = cat_totals[cat_totals['fem_frac'] == 1]
    return cat_fem.sort_values(by=['total'])

In [81]:
# The Red Pill
show_top_fem('TRP')

,fem,masc,neut,it,you,total,fem_frac
word,,,,,,,


In [37]:
# Men's Rights Activists
show_top_fem('MRA')

,fem,masc,neut,it,you,total,fem_frac
word,,,,,,,
happy wife,11,0,0,0,0,11,1.0
next girl,11,0,0,0,0,11,1.0
average girl,11,0,0,0,0,11,1.0
normal woman,11,0,0,0,0,11,1.0
straight woman,11,0,0,0,0,11,1.0
...,...,...,...,...,...,...,...
adult woman,107,0,0,0,0,107,1.0
married woman,115,0,0,0,0,115,1.0
teenage girl,118,0,0,0,0,118,1.0


In [38]:
# Pick Up Artists
show_top_fem('PUA')

,fem,masc,neut,it,you,total,fem_frac
word,,,,,,,
twin sister,11,0,0,0,0,11,1.0
chinese girl,11,0,0,0,0,11,1.0
swedish girl,11,0,0,0,0,11,1.0
better girl,11,0,0,1,0,11,1.0
1 woman,12,0,0,0,0,12,1.0
...,...,...,...,...,...,...,...
gorgeous girl,195,0,0,1,4,195,1.0
attractive woman,201,0,0,0,0,201,1.0
women,214,0,0,36,60,214,1.0


In [39]:
# Incels
show_top_fem('Incels')

,fem,masc,neut,it,you,total,fem_frac
word,,,,,,,
female partner,11,0,0,0,0,11,1.0
current wife,11,0,0,0,0,11,1.0
older lady,11,0,0,0,0,11,1.0
wonderful woman,11,0,0,0,0,11,1.0
current girl,11,0,0,1,0,11,1.0
...,...,...,...,...,...,...,...
right girl,105,0,0,5,0,105,1.0
dream girl,122,0,0,0,0,122,1.0
beautiful girl,131,0,0,1,0,131,1.0


In [40]:
# Men Who Go Their Own Way
show_top_fem('MGTOW')

,fem,masc,neut,it,you,total,fem_frac
word,,,,,,,
crazy girl,11,0,0,0,0,11,1.0
chubby girl,11,0,0,0,0,11,1.0
adult woman,11,0,0,0,0,11,1.0
chic,11,0,0,5,0,11,1.0
ex girlfriends,11,0,0,0,0,11,1.0
...,...,...,...,...,...,...,...
good woman,116,0,0,0,0,116,1.0
attractive woman,131,0,0,0,0,131,1.0
same woman,153,0,0,2,0,153,1.0


In [41]:
# Femcels
show_top_fem('Femcels')

,fem,masc,neut,it,you,total,fem_frac
word,,,,,,,
actress,12,0,0,0,0,12,1.0
hot girl,12,0,0,0,0,12,1.0
beautiful girl,12,0,0,1,0,12,1.0
other woman,13,0,0,0,0,13,1.0
younger sister,13,0,0,0,0,13,1.0
niece,14,0,0,0,0,14,1.0
pretty woman,14,0,0,0,0,14,1.0
girls,14,0,0,5,6,14,1.0
older woman,14,0,0,0,0,14,1.0


In [42]:
# FDS
show_top_fem('FDS')

,fem,masc,neut,it,you,total,fem_frac
word,,,,,,,
young woman,13,0,0,0,0,13,1.0
ex wife,14,0,0,0,0,14,1.0
other woman,14,0,0,1,1,14,1.0
poor woman,16,0,0,0,0,16,1.0


### Popular masc words in reddit

In [43]:
def show_top_masc(cat): 
    cat_df = df[df.category == cat]
    cat_totals = cat_df.groupby('word').sum()
    cat_totals['total'] = cat_totals['fem'] + cat_totals['masc'] 
    # filter to only those that appear at least 10 times as she or he
    cat_totals = cat_totals[cat_totals['total'] > 10] 
    cat_totals['masc_frac'] = cat_totals['masc'] / (cat_totals['fem'] + cat_totals['masc'])
    cat_masc = cat_totals[cat_totals['masc_frac'] == 1]
    return cat_masc.sort_values(by=['total'])

In [44]:
# The Red Pill
show_top_masc('TRP')

,fem,masc,neut,it,you,total,masc_frac
word,,,,,,,
mailman,0,11,0,2,0,11,1.0
manipulated man,0,11,0,0,0,11,1.0
frat guy,0,11,0,0,0,11,1.0
old dude,0,11,0,3,0,11,1.0
family alpha,0,11,0,3,0,11,1.0
...,...,...,...,...,...,...,...
successful man,0,84,0,0,0,84,1.0
short guy,0,86,0,3,0,86,1.0
coach,0,125,0,4,0,125,1.0


In [45]:
# MRA
show_top_masc('MRA')

,fem,masc,neut,it,you,total,masc_frac
word,,,,,,,
tall man,0,11,0,0,0,11,1.0
white kid,0,11,0,0,0,11,1.0
wise man,0,11,0,0,0,11,1.0
fat guy,0,11,0,2,0,11,1.0
blind man,0,11,0,0,0,11,1.0
...,...,...,...,...,...,...,...
random guy,0,57,0,0,0,57,1.0
first guy,0,60,0,2,0,60,1.0
good man,0,75,0,0,0,75,1.0


In [46]:
# PUA
show_top_masc('PUA')

,fem,masc,neut,it,you,total,masc_frac
word,,,,,,,
rich man,0,11,0,0,1,11,1.0
best buddy,0,11,0,0,0,11,1.0
nerdy guy,0,11,0,0,0,11,1.0
big guy,0,11,0,1,0,11,1.0
billionaire,0,11,0,4,0,11,1.0
little boy,0,11,0,0,0,11,1.0
autistic guy,0,12,0,0,0,12,1.0
different guy,0,12,0,0,1,12,1.0
good wingman,0,12,0,2,0,12,1.0


In [47]:
# Incels
show_top_masc('Incels')

,fem,masc,neut,it,you,total,masc_frac
word,,,,,,,
male incel,0,11,0,2,0,11,1.0
everybody,0,11,0,0,1,11,1.0
drunk guy,0,11,0,0,0,11,1.0
dead man,0,11,0,0,0,11,1.0
current bf,0,11,0,8,0,11,1.0
...,...,...,...,...,...,...,...
homeless guy,0,59,0,2,0,59,1.0
right guy,0,90,0,2,0,90,1.0
gay guy,0,96,0,1,0,96,1.0


In [48]:
# MGTOW
show_top_masc('MGTOW')

,fem,masc,neut,it,you,total,masc_frac
word,,,,,,,
younger self,0,11,0,0,1,11,1.0
bad man,0,11,0,0,0,11,1.0
short guy,0,11,0,0,0,11,1.0
honest man,0,11,0,0,1,11,1.0
happy man,0,12,0,0,0,12,1.0
...,...,...,...,...,...,...,...
hero,0,53,0,17,0,53,1.0
innocent man,0,67,0,0,0,67,1.0
successful man,0,68,0,0,0,68,1.0


In [49]:
# Femcels
show_top_masc('Femcels')

,fem,masc,neut,it,you,total,masc_frac
word,,,,,,,
black man,0,11,0,0,0,11,1.0
first guy,0,12,0,0,0,12,1.0
same guy,0,12,0,0,0,12,1.0
average man,0,15,0,0,0,15,1.0
ugly guy,0,15,0,0,0,15,1.0
ugly man,0,15,0,0,0,15,1.0
average guy,0,16,0,0,0,16,1.0
other guy,0,20,0,1,0,20,1.0
son,0,45,0,0,1,45,1.0


In [50]:
# FDS
show_top_masc('FDS')

,fem,masc,neut,it,you,total,masc_frac
word,,,,,,,
current partner,0,11,0,1,0,11,1.0
good guy,0,11,0,0,0,11,1.0
ugly guy,0,15,0,0,0,15,1.0
good man,0,16,0,0,0,16,1.0
married man,0,16,0,0,0,16,1.0
right man,0,16,0,0,0,16,1.0
new guy,0,19,0,0,0,19,1.0
bf,0,49,0,30,0,49,1.0
one guy,0,49,0,0,0,49,1.0


### Popular neut words in reddit

In [51]:
def show_top_neut(cat): 
    cat_df = df[df.category == cat]
    cat_totals = cat_df.groupby('word').sum()
    cat_totals['total'] = cat_totals['fem'] + cat_totals['masc'] 
    # filter to only those that appear at least 10 times as she or he
    cat_totals = cat_totals[cat_totals['total'] > 10] 
    cat_totals['fem_frac'] = cat_totals['fem'] / (cat_totals['fem'] + cat_totals['masc'])
    cat_neut = cat_totals[cat_totals.fem_frac.between(0.45, 0.55)]
    return cat_neut.sort_values(by=['total'], ascending = False)

In [52]:
# The Red Pill
show_top_neut('TRP')

,fem,masc,neut,it,you,total,fem_frac
word,,,,,,,
baby,727,733,0,632,8,1460,0.497945
roommate,389,456,0,10,1,845,0.460355
teacher,353,376,0,1,4,729,0.484225
asshole,202,201,0,612,3,403,0.501241
victim,202,194,0,206,1,396,0.510101
attorney,117,109,0,10,0,226,0.517699
beta bux,78,91,0,24,3,169,0.461538
neighbor,72,78,0,19,0,150,0.480000
stranger,73,64,0,53,1,137,0.532847


In [53]:
show_top_neut('MRA')

,fem,masc,neut,it,you,total,fem_frac
word,,,,,,,
friend,2537,2862,0,10,96,5399,0.469902
victim,1345,1448,0,1166,2,2793,0.481561
author,1329,1415,0,305,1,2744,0.484329
rapist,611,742,0,365,1,1353,0.451589
partner,572,628,0,186,2,1200,0.476667
professor,304,314,0,24,1,618,0.491909
writer,262,239,0,78,1,501,0.522954
cop,113,114,0,223,0,227,0.497797
roommate,95,112,0,0,2,207,0.458937


In [54]:
show_top_neut('PUA')

,fem,masc,neut,it,you,total,fem_frac
word,,,,,,,
friend,8014,6655,0,57,403,14669,0.546322
boyfriend,1165,1044,0,51,12,2209,0.527388
person,993,1137,0,161,26,2130,0.466197
best friend,372,425,0,0,23,797,0.466750
cousin,204,234,0,1,0,438,0.465753
bro,126,129,0,34,21,255,0.494118
teacher,102,86,0,0,2,188,0.542553
good friend,83,75,0,0,1,158,0.525316
someone else,71,84,0,14,14,155,0.458065


In [55]:
show_top_neut('Incels')

,fem,masc,neut,it,you,total,fem_frac
word,,,,,,,
ex,447,515,0,89,2,962,0.464657
teacher,254,239,0,0,1,493,0.515213
victim,83,84,0,103,0,167,0.497006
psychologist,56,67,0,6,0,123,0.455285
good friend,54,63,0,0,2,117,0.461538
classmate,56,60,0,0,0,116,0.482759
stranger,53,58,0,76,2,111,0.477477
colleague,46,50,0,0,1,96,0.479167
supervisor,32,34,0,2,1,66,0.484848


In [56]:
show_top_neut('MGTOW')

,fem,masc,neut,it,you,total,fem_frac
word,,,,,,,
boyfriend,639,723,0,26,5,1362,0.469163
baby,567,601,0,585,3,1168,0.485445
coworker,246,218,0,5,2,464,0.530172
partner,241,210,0,55,0,451,0.534368
neighbor,77,84,0,13,0,161,0.478261
asshole,78,80,0,178,1,158,0.493671
attorney,73,80,0,17,1,153,0.477124
writer,56,66,0,10,0,122,0.459016
adult,63,54,0,26,1,117,0.538462


In [59]:
show_top_neut('Femcels')

,fem,masc,neut,it,you,total,fem_frac
word,,,,,,,
partner,58,58,0,31,0,116,0.500000
child,30,32,0,16,0,62,0.483871
teacher,26,25,0,0,0,51,0.509804
looksmatch,12,13,0,12,0,25,0.480000
sis,11,11,0,4,2,22,0.500000
incels,9,11,0,91,0,20,0.450000
classmate,6,6,0,0,0,12,0.500000
good friend,6,5,0,0,0,11,0.545455


In [60]:
show_top_neut('FDS')

,fem,masc,neut,it,you,total,fem_frac
word,,,,,,,
asshole,7,7,0,8,0,14,0.5


### Gender over time 

In [63]:
# These are the words whose % fem ranges over time are the largest,
# maybe top three words with biggest % range in each community, e.g. “cat	10% - 50%”, only calculate fraction 
# if there are more than 10 occurrences in each community and month. 

def show_top_fem_range(cat):
    # filtering for the argument category
    cat_df = df[df.category == cat]
    totals = cat_df.groupby(['month', 'word'], as_index = False).sum()
    totals['total'] = totals['fem'] + totals['masc']
    totals = totals[totals['total'] > 10]
    totals['fem_frac'] = totals['fem'] / (totals['fem'] + totals['masc'])
    
    # filter for words that show up in more than 1 of the months/time periods 
    # (initially picked 85 to be more than half of all the months but idk if needed)
    is_multi = totals["word"].value_counts() > 1
    filtered = totals[totals["word"].isin(is_multi[is_multi].index)]
    
    # get the max and min fem_frac for each word
    word_keys = filtered['word'].unique().tolist()
    max_fems = []
    min_fems = []
    max_months = []
    min_months = []
    for word in word_keys: 
        df_subset = filtered[filtered['word'] == word]
        max_fem = df_subset['fem_frac'].max()
        min_fem = df_subset['fem_frac'].min()
        max_month = df_subset[df_subset['fem_frac'] == max_fem]['month'].max()
        min_month = df_subset[df_subset['fem_frac'] == min_fem]['month'].min()
        
        max_fems.append(max_fem)
        min_fems.append(min_fem)
        max_months.append(max_month)
        min_months.append(min_month)
    
    
    d = {'word': [], 'min month': [], 'min': [], 'max month': [], 'max':[], 'diff': []}
    for i in range(len(word_keys)):
        d['word'].append(word_keys[i])
        d['min month'].append(min_months[i])
        d['min'].append(min_fems[i])
        d['max month'].append(max_months[i])
        d['max'].append(max_fems[i])
        d['diff'].append(max_fems[i] - min_fems[i])
    
    diffs = pd.DataFrame(data=d)
    return diffs.sort_values(by = ['diff'], ascending = False)


In [64]:
# words in each month in Incels that appear more than 10 times in that month
show_top_fem_range("Incels")

,word,min month,min,max month,max,diff
5,girlfriend,2019-04,0.042298,2019-07,1.000000,0.957702
30,roommate,2018-06,0.000000,2017-06,0.714286,0.714286
10,ex,2018-01,0.214286,2011-09,0.916667,0.702381
24,coworker,2019-04,0.333333,2017-06,0.937500,0.604167
29,partner,2018-11,0.166667,2019-12,0.750000,0.583333
...,...,...,...,...,...,...
70,average woman,2018-04,1.000000,2018-10,1.000000,0.000000
68,ugly man,2018-03,0.000000,2018-06,0.000000,0.000000
64,same girl,2017-10,1.000000,2018-05,1.000000,0.000000
42,cute girl,2016-12,1.000000,2018-11,1.000000,0.000000


In [65]:
show_top_fem_range("TRP")

,word,min month,min,max month,max,diff
48,teacher,2017-05,0.000000,2014-10,0.823529,0.823529
49,fiance,2018-09,0.185185,2019-09,1.000000,0.814815
69,coworker,2015-08,0.142857,2016-08,0.916667,0.773810
55,professor,2016-02,0.090909,2015-12,0.769231,0.678322
41,doctor,2013-11,0.000000,2019-01,0.666667,0.666667
...,...,...,...,...,...,...
50,good man,2014-01,0.000000,2018-12,0.000000,0.000000
87,next girl,2014-10,1.000000,2018-06,1.000000,0.000000
63,average woman,2014-03,1.000000,2017-10,1.000000,0.000000
71,good woman,2014-04,1.000000,2015-01,1.000000,0.000000


In [66]:
show_top_fem_range("MRA")

,word,min month,min,max month,max,diff
26,teacher,2014-09,0.250000,2019-12,1.000000,0.750000
32,rapist,2013-05,0.083333,2017-07,0.800000,0.716667
40,officer,2012-10,0.000000,2019-12,0.714286,0.714286
31,partner,2013-03,0.117647,2015-07,0.785714,0.668067
25,ex,2011-12,0.347826,2017-06,1.000000,0.652174
22,lawyer,2017-11,0.000000,2018-08,0.611111,0.611111
18,baby,2017-03,0.096774,2016-01,0.687500,0.590726
16,author,2012-03,0.217391,2018-04,0.785714,0.568323
35,doctor,2013-10,0.000000,2011-08,0.545455,0.545455
53,professor,2013-11,0.266667,2017-03,0.800000,0.533333


In [67]:
show_top_fem_range("PUA")

,word,min month,min,max month,max,diff
8,someone,2011-04,0.086957,2018-02,0.750000,0.663043
33,ex,2013-03,0.266667,2012-08,0.916667,0.650000
11,person,2013-07,0.142857,2017-04,0.750000,0.607143
4,boyfriend,2017-07,0.235294,2019-03,0.833333,0.598039
12,roommate,2010-11,0.181818,2012-06,0.727273,0.545455
14,dude,2019-12,0.000000,2019-05,0.500000,0.500000
39,coworker,2019-04,0.461538,2017-06,0.916667,0.455128
26,best friend,2012-05,0.272727,2013-01,0.666667,0.393939
9,buddy,2012-06,0.000000,2013-07,0.370370,0.370370
27,brother,2011-07,0.000000,2017-08,0.363636,0.363636


In [68]:
show_top_fem_range("MGTOW")

,word,min month,min,max month,max,diff
30,author,2018-06,0.090909,2018-11,0.785714,0.694805
59,partner,2019-07,0.222222,2017-12,0.875000,0.652778
70,bro,2019-07,0.000000,2018-04,0.636364,0.636364
32,cousin,2016-10,0.260870,2017-10,0.785714,0.524845
63,coworker,2018-12,0.230769,2019-07,0.714286,0.483516
...,...,...,...,...,...,...
78,prince,2018-05,0.260870,2018-12,0.272727,0.011858
77,average woman,2018-04,1.000000,2018-12,1.000000,0.000000
84,little girl,2018-11,1.000000,2018-12,1.000000,0.000000
53,old man,2017-07,0.000000,2019-05,0.000000,0.000000


In [69]:
show_top_fem_range("MGTOW")

,word,min month,min,max month,max,diff
30,author,2018-06,0.090909,2018-11,0.785714,0.694805
59,partner,2019-07,0.222222,2017-12,0.875000,0.652778
70,bro,2019-07,0.000000,2018-04,0.636364,0.636364
32,cousin,2016-10,0.260870,2017-10,0.785714,0.524845
63,coworker,2018-12,0.230769,2019-07,0.714286,0.483516
...,...,...,...,...,...,...
78,prince,2018-05,0.260870,2018-12,0.272727,0.011858
77,average woman,2018-04,1.000000,2018-12,1.000000,0.000000
84,little girl,2018-11,1.000000,2018-12,1.000000,0.000000
53,old man,2017-07,0.000000,2019-05,0.000000,0.000000


In [70]:
show_top_fem_range("Femcels")

,word,min month,min,max month,max,diff
21,partner,2019-01,0.416667,2019-08,1.000000,0.583333
14,person,2019-07,0.000000,2019-04,0.529412,0.529412
6,friend,2018-07,0.312500,2018-09,0.807692,0.495192
19,boyfriend,2019-03,0.058824,2019-05,0.391304,0.332481
9,someone,2018-10,0.050000,2019-03,0.333333,0.283333
3,ex,2018-12,0.454545,2018-06,0.666667,0.212121
12,dad,2019-01,0.000000,2019-07,0.187500,0.187500
15,dude,2019-03,0.000000,2018-11,0.166667,0.166667
20,girlfriend,2019-04,0.833333,2019-11,1.000000,0.166667
7,mother,2019-12,0.833333,2019-11,1.000000,0.166667


In [71]:
show_top_fem_range("FDS")

,word,min month,min,max month,max,diff
5,dude,2019-10,0.000000,2019-11,0.179104,0.179104
2,girl,2019-07,0.833333,2019-10,1.000000,0.166667
12,girlfriend,2019-11,0.791667,2019-10,0.941176,0.149510
9,dad,2019-10,0.055556,2019-12,0.158879,0.103323
8,boyfriend,2019-10,0.115385,2019-11,0.216667,0.101282
27,father,2019-12,0.000000,2019-11,0.100000,0.100000
17,person,2019-10,0.000000,2019-12,0.095238,0.095238
16,partner,2019-11,0.086957,2019-12,0.178571,0.091615
7,friend,2019-11,0.609375,2019-09,0.692308,0.082933
28,kid,2019-12,0.086957,2019-11,0.166667,0.079710


### Popular words in control

In [53]:
control_df = pd.read_csv('pronoun_control_df.csv')

def show_top_fem_control(): 
    control_totals = control_df.groupby('word').sum()
    control_totals['total'] = control_totals['fem'] + control_totals['masc'] 
    # filter to only those that appear more than 10 times as she or he
    control_totals = control_totals[control_totals['total'] > 10] 
    control_totals['fem_frac'] = control_totals['fem'] / (control_totals['fem'] + control_totals['masc'])
    control_fem = control_totals[control_totals['fem_frac'] == 1]
    return control_fem.sort_values(by=['total'], ascending = False)

def show_top_masc_control(): 
    control_totals = control_df.groupby('word').sum()
    control_totals['total'] = control_totals['fem'] + control_totals['masc'] 
    # filter to only those that appear at least 10 times as she or he
    control_totals = control_totals[control_totals['total'] > 10] 
    control_totals['masc_frac'] = control_totals['masc'] / (control_totals['fem'] + control_totals['masc'])
    control_masc = control_totals[control_totals['masc_frac'] == 1]
    return control_masc.sort_values(by=['total'], ascending = False)

def show_top_neut_control(): 
    control_totals = control_df.groupby('word').sum()
    control_totals['total'] = control_totals['fem'] + control_totals['masc'] 
    # filter to only those that appear at least 10 times as she or he
    control_totals = control_totals[control_totals['total'] > 10] 
    control_totals['fem_frac'] = control_totals['fem'] / (control_totals['fem'] + control_totals['masc'])
    control_neut = control_totals[control_totals['fem_frac'].between(0.45, 0.55)]
    return control_neut.sort_values(by=['total'], ascending = False)

In [ ]:
show_top_neut_control()

### Gender differences

In [54]:
# len(df.word.unique()
df = df.groupby('word').sum()
df['total'] = df['fem'] + df['masc']
df = df[df['total'] > 10] 
df['fem_frac'] = df['fem'] / (df['fem'] + df['masc'])
df = df.sort_values(by=['fem'], ascending = False)
df


,fem,masc,neut,total,fem_frac
word,,,,,
girl,308205,2567,226,310772,0.991740
woman,248514,1024,838,249538,0.995896
wife,74069,576,55,74645,0.992283
mother,31271,275,139,31546,0.991283
mom,23830,893,186,24723,0.963880
...,...,...,...,...,...
physicist,0,29,1,29,0.000000
tall man,0,59,1,59,0.000000
soccer player,0,11,0,11,0.000000


In [56]:
control_df = control_df.groupby('word').sum()
control_df['total'] = control_df['fem'] + control_df['masc']
control_df = control_df[control_df['total'] > 10] 
control_df['fem_frac'] = control_df['fem'] / (control_df['fem'] + control_df['masc'])
control_df = control_df.sort_values(by=['fem'], ascending = False)
control_df


,fem,masc,neut,total,fem_frac
word,,,,,
mom,22693,764,200,23457,0.967430
wife,22389,165,18,22554,0.992684
girl,22147,269,29,22416,0.988000
woman,17238,131,56,17369,0.992458
mother,13385,107,53,13492,0.992069
...,...,...,...,...,...
composer,0,29,7,29,0.000000
common man,0,34,1,34,0.000000
colonel,0,25,0,25,0.000000


In [57]:
merged_df = df.merge(control_df, how='inner', left_index=True, right_index=True)
merged_df

,fem_x,masc_x,neut_x,total_x,fem_frac_x,fem_y,masc_y,neut_y,total_y,fem_frac_y
word,,,,,,,,,,
girl,308205,2567,226,310772,0.991740,22147,269,29,22416,0.988000
woman,248514,1024,838,249538,0.995896,17238,131,56,17369,0.992458
wife,74069,576,55,74645,0.992283,22389,165,18,22554,0.992684
mother,31271,275,139,31546,0.991283,13385,107,53,13492,0.992069
mom,23830,893,186,24723,0.963880,22693,764,200,23457,0.967430
...,...,...,...,...,...,...,...,...,...,...
layman,0,11,0,11,0.000000,1,11,0,12,0.083333
first man,0,69,0,69,0.000000,0,31,0,31,0.000000
physicist,0,29,1,29,0.000000,1,22,2,23,0.043478


In [59]:
merged_df['difference'] = (merged_df['fem_frac_x'] - merged_df['fem_frac_y']).abs()
merged_df = merged_df.sort_values(by=['difference'], ascending = False)

In [61]:
merged_df

,fem_x,masc_x,neut_x,total_x,fem_frac_x,fem_y,masc_y,neut_y,total_y,fem_frac_y,difference
word,,,,,,,,,,,
brat,21,5,1,26,0.807692,2,10,1,12,0.166667,0.641026
expert,68,35,25,103,0.660194,8,48,31,56,0.142857,0.517337
band,15,2,382,17,0.882353,8,13,1799,21,0.380952,0.501401
cunt,789,205,60,994,0.793763,35,77,12,112,0.312500,0.481263
sitter,11,3,1,14,0.785714,7,14,0,21,0.333333,0.452381
...,...,...,...,...,...,...,...,...,...,...,...
trans woman,131,0,1,131,1.000000,73,0,4,73,1.000000,0.000000
grown woman,120,0,5,120,1.000000,32,0,0,32,1.000000,0.000000
own daughter,112,0,0,112,1.000000,48,0,0,48,1.000000,0.000000


### Pronoun sparsity

In [84]:
df = pd.read_csv('pronoun_df.csv')
# df = df.groupby('word').sum()
# df.shape[0]

# total vocab words that show up in reddit with masc/fem/neut pronouns is 6373

df_totals = df.groupby('word').sum()
# df_totals['total'] = df_totals['fem'] + df_totals['masc'] 
df_totals['total'] = df_totals['fem'] + df_totals['masc'] + df_totals['neut']

df_totals['neut_frac'] = df_totals['neut'] / (df_totals['fem'] + df_totals['masc'] + df_totals['neut'])
df_totals 

df_neut = df_totals[df_totals['neut_frac'] >= 0.5].sort_values(by = ['neut_frac'], ascending = False)
# df_neut.head(20)
df_neut

# df_sparse = df_totals[df_totals['total'] <= 10]
# df_sparse



# 4300 words have less than 10 occurrences with masc or fem pronouns => ~70%
# 1438 words have less than 10 occurrences with masc or fem or neut pronouns => ~23%

# 3716 words are mostly "they" words => ~60%



,fem,masc,neut,total,neut_frac
word,,,,,
zombies,0,0,97,97,1.0
monarchs,0,0,18,18,1.0
mobile users,0,0,1,1,1.0
mockingbird,0,0,1,1,1.0
moderate feminists,0,0,86,86,1.0
...,...,...,...,...,...
walking wallet,2,0,2,4,0.5
total loser,0,2,2,4,0.5
changs,0,2,2,4,0.5
